In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,013 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-27 16:47:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-06-27 16:47:16 (10.6 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql.functions import count, avg, max, min, mean
# Read in the Review dataset as a DataFrame

# convert the review_date to a date type
reviews_df = df.withColumn("review_date", to_date(col("review_date"), "yyyy-MM-dd"))
reviews_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

In [7]:
reviews_df.printSchema()
reviews_df.count()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



6221559

In [8]:
# Create the customers_table DataFrame
customers_table = reviews_df.groupby("customer_id").agg(count("customer_id")).withColumnRenamed("count(customer_id)", "customer_count")
# reviews_df.groupBy("customer_id").count().show()
customers_table.show()
customers_table.count()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28241302|             1|
|   15991768|             1|
|   17067926|             7|
|    8905239|             3|
|   51794064|             1|
|   33132797|             3|
|   21325555|             1|
|    1574612|             1|
|   12819130|             3|
|   44224073|             1|
|   16679709|             1|
|   10742726|             1|
|   52391013|             2|
|   38209321|            10|
|    2545142|             1|
|   52918362|             6|
|   47663761|             3|
|   25031530|            21|
|   29476112|             1|
|   51496479|             1|
+-----------+--------------+
only showing top 20 rows



3467883

In [9]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0486417700|Metallic Foil Ori...|
|1619922363|Cavallini 2014 Fl...|
|1619924307|Cavallini Papers ...|
|1842104497|HUGH JACKMAN 2014...|
|7078976541|VECTRA® 22 Fabric...|
|7482784535|Genuine Leather C...|
|753854514X|Low Voltage 12V A...|
|9178889472|Småland Böna IKEA...|
|9178892171|IKEA - ROTERA Lan...|
|9178909309|IKEA - PORTIS Hat...|
|B00004W40I|King Electrical 6...|
|B00005QFT1|Le Creuset Jumbo ...|
|B00005T3Z7|Endust for Electr...|
|B00006498M|SINGER 6038 46-St...|
|B0000665TD|J-2000 Garment St...|
|B000069BF9|Euro-Pro Shark EP...|
|B00006GXAP|  Ecru Squares Quilt|
|B000075MI0|Stainless Steel C...|
|B000095ZGF|Polder 90616-05 W...|
|B000096P32|Testors Enamel Fi...|
+----------+--------------------+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import countDistinct
products_df.select(countDistinct("product_id")).show()

+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                   1074856|
+--------------------------+



In [10]:
products_df.count()

1074856

In [14]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = reviews_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()
review_id_df.printSchema()
review_id_df.count()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1UUISQ1GKOJTI|   33670092|B00EE62UAE|     583436067| 2015-08-31|
|R1HOJ9WE8VCVOD|   13726692|B001APXO5C|     465035091| 2015-08-31|
| RDNGVXMWQN2TN|   50131396|B002HFDLCK|     136507891| 2015-08-31|
|R3OM9COQMVTDJ2|   16046884|B00PL9EFPQ|     631238459| 2015-08-31|
|R3OFUQVR4Y80Q9|   11417282|B00B5TPVQS|     190431573| 2015-08-31|
|R2GD25SBBGRKPO|   48013430|B00NQDGHDC|     124050883| 2015-08-31|
|R3RRSLYKVWH9WB|    7341516|B00I3BYEXM|     933053612| 2015-08-31|
|R2HMFAGJJU6NT3|   20696954|B008QZD7RY|      77515396| 2015-08-31|
|R2QDRZATHBY4GY|   28241302|B00TQ6MXE0|     267158600| 2015-08-31|
|R107946YZK57Q2|   45444347|B004O39RJ4|     473994651| 2015-08-31|
|R31EK6FUI5YAL1|   15080335|B003LZ09C0|     945333576| 2015-08-31|
|R2PCO0R2FKDQLD|   12081067|B00I56KQV4|     383995956| 2015-08

6221559

In [15]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()
vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1UUISQ1GKOJTI|          1|            0|          0|   N|                Y|
|R1HOJ9WE8VCVOD|          5|            9|          9|   N|                Y|
| RDNGVXMWQN2TN|          5|            0|          0|   N|                Y|
|R3OM9COQMVTDJ2|          2|            0|          0|   N|                Y|
|R3OFUQVR4Y80Q9|          4|            0|          0|   N|                N|
|R2GD25SBBGRKPO|          5|            0|          0|   N|                Y|
|R3RRSLYKVWH9WB|          5|            0|          0|   N|                Y|
|R2HMFAGJJU6NT3|          5|            0|          0|   N|                Y|
|R2QDRZATHBY4GY|          1|            0|          0|   N|                Y|
|R107946YZK57Q2|          2|            0|          0|   N|     

6221559

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [16]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://sellbyhomereviews.czvjnaxvazlq.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"root", 
          "password": "bullcity9!9", 
          "driver":"org.postgresql.Driver"}

In [17]:
# Write review_id_df to table in RDS
# review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [18]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [19]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_table.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [20]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)